In [ ]:
import os
import re
import requests
from pathlib import Path
from typing import List, Tuple


def find_markdown_files(directory: Path) -> List[Path]:
    """
    Recursively find all Markdown files in the given directory.

    Args:
        directory (Path): The directory to search.

    Returns:
        List[Path]: A list of paths to Markdown files.
    """
    return list(directory.rglob("*.md"))


def extract_image_urls(markdown_content: str) -> List[str]:
    """
    Extract all image URLs from the Markdown content, including those without file extensions.

    Args:
        markdown_content (str): The content of a Markdown file.

    Returns:
        List[str]: A list of image URLs.
    """
    # Match standard image URLs with extensions (jpg, png)
    urls_with_extensions = re.findall(r'!\[.*?\]\((https?://.*?\.(?:jpg|png))\)', markdown_content)
    # Match URLs from blogger.googleusercontent.com (no extension)
    blogger_urls = re.findall(r'!\[.*?\]\((https?://blogger\.googleusercontent\.com/img/.*?)\)', markdown_content)
    return urls_with_extensions + blogger_urls


def download_image(url: str, destination: Path) -> None:
    """
    Download an image from a URL and save it to the destination.

    Args:
        url (str): The URL of the image.
        destination (Path): The path to save the image.
    """
    try:
        response = requests.get(url, stream=True, timeout=10)
        response.raise_for_status()
        with open(destination, "wb") as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)
    except Exception as e:
        log_file = Path("error_log.txt")
        with log_file.open("a", encoding="utf-8") as log:
            log.write(f"Failed to download {url}: {e}\n")
        print(f"Error downloading {url}. Logged to {log_file}.")

def replace_image_urls(markdown_content: str, url_map: List[Tuple[str, str]]) -> str:
    """
    Replace image URLs in the Markdown content with local paths.

    Args:
        markdown_content (str): The content of a Markdown file.
        url_map (List[Tuple[str, str]]): A list of tuples mapping old URLs to new local paths.

    Returns:
        str: The updated Markdown content.
    """
    for old_url, new_path in url_map:
        markdown_content = markdown_content.replace(old_url, new_path)
    return markdown_content


def make_short_name(seed: int = 4) -> str:
    """generate a short name with a random number
    args:
        seed (int): The seed for random number generation.
    """
    return  str(int.from_bytes(os.urandom(seed), 'big'))


def process_markdown_files(directory: Path, assets_folder: Path) -> None:
    """
    Process all Markdown files in the directory:
    - Extract image URLs.
    - Download images to the assets folder.
    - Replace URLs in the Markdown files with local paths.

    Args:
        directory (Path): The directory containing Markdown files.
        assets_folder (Path): The folder to store downloaded images.
    """
    assets_folder.mkdir(parents=True, exist_ok=True)
    md_files = find_markdown_files(directory)
    print(f"found {len(md_files)} markdown files")
    for markdown_file in md_files:
        print(f"Processing {markdown_file}")
        with open(markdown_file, "r", encoding="utf-8") as file:
            content = file.read()

        image_urls = extract_image_urls(content)
        print(f"found {len(image_urls)} images in {markdown_file.name}")
        url_map = []

        for url in image_urls:
            # Generate a unique name for the image
            image_name = url.split("/")[-1]
            if not any(image_name.endswith(ext) for ext in [".jpg", ".png"]):
                image_name += ".jpg"  # Default to .jpg if no extension is present
            if len(image_name) > 20:
                image_name = make_short_name() + ".jpg"
            local_path = assets_folder / image_name
            if not local_path.exists():
                download_image(url, local_path)
            url_map.append((url, str(local_path.relative_to(directory))))

        updated_content = replace_image_urls(content, url_map)

        with open(markdown_file, "w", encoding="utf-8") as file:
            file.write(updated_content)


if __name__ == "__main__":
    # Define the directory containing Markdown files and the assets folder
    markdown_directory = Path().cwd()
    assets_directory = markdown_directory / "pics"

    # Process the Markdown files
    process_markdown_files(markdown_directory, assets_directory)

found 68 markdown files
Processing /home/richard/programmieren/spurtreu/posts/2022-08-27-wie-reist-man-heutzutage.md
found 0 images in 2022-08-27-wie-reist-man-heutzutage.md
Processing /home/richard/programmieren/spurtreu/posts/2023-05-01-heraus-zum-ersten-mai.md
found 3 images in 2023-05-01-heraus-zum-ersten-mai.md


ValueError: '/home/richard/programmieren/spurtreu/assets/1122622944.jpg' is not in the subpath of '/home/richard/programmieren/spurtreu/posts'

In [5]:
str(int.from_bytes(os.urandom(4), 'big'))

'4228820022'